In [93]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import sqlite3 


In [44]:
"""Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
Rename the columns as per the description from this file:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Task: Create a sql db from adult dataset and name it sqladb
"""

df = pd.read_csv('adult.data',header = None)


df.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','>50K, <=50K']

df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K"
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [70]:
conn = sqlite3.connect('sqladb')
cur=conn.cursor()
cur.execute("drop table if exists Adult; ")


In [71]:
df.to_sql('Adult', conn, if_exists='replace', index=True) 


In [72]:
"""1 1.Select 10 records from the adult sqladb
"""

pd.read_sql('select * from Adult limit 10 ', conn)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K"
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [85]:
"""2.Show me the average hours per week of all men who are working in private
"""

q2 = 'select cast(sum("hours-per-week") as float)/cast(count(*) as float),avg("hours-per-week") from Adult where upper(ltrim(rtrim("sex"))) = "MALE" and upper(ltrim(rtrim("workclass"))) = "PRIVATE"'

pd.read_sql(q2,conn)

,"cast(sum(""hours-per-week"") as float)/cast(count(*) as float)","avg(""hours-per-week"")"
0,42.221226,42.221226


In [86]:
"""
3.Show me the frequency table for education, occupation and relationship,
separately
'"""

q2 = 'select count(*),"education" from Adult group by "education"'

pd.read_sql(q2,conn)

,count(*),education
0,933,10th
1,1175,11th
2,433,12th
3,168,1st-4th
4,333,5th-6th
5,646,7th-8th
6,514,9th
7,1067,Assoc-acdm
8,1382,Assoc-voc
9,5355,Bachelors


In [82]:

q2 = 'select count(*),"occupation" from Adult group by "occupation"'

pd.read_sql(q2,conn)

,count(*),occupation
0,1843,?
1,3770,Adm-clerical
2,9,Armed-Forces
3,4099,Craft-repair
4,4066,Exec-managerial
5,994,Farming-fishing
6,1370,Handlers-cleaners
7,2002,Machine-op-inspct
8,3295,Other-service
9,149,Priv-house-serv


In [83]:

q2 = 'select count(*),"relationship" from Adult group by "relationship"'

pd.read_sql(q2,conn)

,count(*),relationship
0,13193,Husband
1,8305,Not-in-family
2,981,Other-relative
3,5068,Own-child
4,3446,Unmarried
5,1568,Wife


In [88]:
"""
4.Are there any people who are married, working in private sector and having a
master’s degree
"""


q2 = 'select case when cnt > 0 then "Yes there are!" else "No" end Exists_Check , cnt from (select count(*) as cnt from Adult where ltrim(rtrim(upper("relationship"))) in ("HUSBAND","WIFE") and upper(ltrim(rtrim("workclass"))) = "PRIVATE" and upper(ltrim(rtrim("education"))) = "MASTERS")abc'

pd.read_sql(q2,conn)

,Exists_Check,cnt
0,Yes there are!,528


In [89]:
"""
5. What is the average, minimum and maximum age group for people working in
different sectors
"""

q2 = 'select avg(age),min(age),max(age),workclass from Adult group by workclass'

pd.read_sql(q2,conn)


,avg(age),min(age),max(age),workclass
0,40.960240,17,90,?
1,42.590625,17,90,Federal-gov
2,41.751075,17,90,Local-gov
3,20.571429,17,30,Never-worked
4,36.797585,17,90,Private
5,46.017025,17,84,Self-emp-inc
6,44.969697,17,90,Self-emp-not-inc
7,39.436055,17,81,State-gov
8,47.785714,19,72,Without-pay


In [90]:
"""
6. Calculate age distribution by country

"""

q2='SELECT SUM(CASE WHEN AGE < 18 THEN 1 ELSE 0 END) AS [UNDER 18],     SUM(CASE WHEN AGE BETWEEN 18 AND 24 THEN 1 ELSE 0 END) AS [18-24],        SUM(CASE WHEN AGE BETWEEN 25 AND 34 THEN 1 ELSE 0 END) AS [25-34],		"NATIVE-COUNTRY"		FROM ADULT GROUP BY "NATIVE-COUNTRY"		'
pd.read_sql(q2,conn)


,UNDER 18,18-24,25-34,native-country
0,2,66,166,?
1,0,1,6,Cambodia
2,2,11,29,Canada
3,0,5,20,China
4,0,6,21,Columbia
5,0,5,16,Cuba
6,0,14,18,Dominican-Republic
7,0,4,12,Ecuador
8,2,26,34,El-Salvador
9,1,10,25,England


In [91]:
"""
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-
gain' and 'capital-loss'"""

q2='SELECT distinct "capital-gain", "capital-loss",cast(("capital-gain" - "capital-loss") as float)/case when cast(("capital-loss") as float) = 0 then 1 else cast(("capital-loss") as float) end  as "Net-Capital-Gain" from Adult order by cast(("capital-gain" - "capital-loss") as float)/cast(("capital-loss") as float) desc '
pd.read_sql(q2,conn)


,capital-gain,capital-loss,Net-Capital-Gain
0,0,2042,-1.0
1,0,1408,-1.0
2,0,1902,-1.0
3,0,1573,-1.0
4,0,1887,-1.0
5,0,1719,-1.0
6,0,1762,-1.0
7,0,1564,-1.0
8,0,2179,-1.0
9,0,1816,-1.0


In [92]:
conn.commit()
conn.close()

In [1]:
"""Problem statement 2:
Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Task:

1. Create an sqlalchemy engine using a sample from the data set
2. Write two basic update queries
3. Write two delete queries
4. Write two filter queries
5. Write two function queries
Note: Solution submitted via github"""

import sqlalchemy as db

engine = db.create_engine('sqlite:///sqladb')
connection = engine.connect()
metadata = db.MetaData()
Adult = db.Table('Adult', metadata, autoload=True, autoload_with=engine)

print(Adult.columns.keys())



['index', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', '>50K, <=50K']


In [6]:
"1) "

import sqlalchemy as db

engine = db.create_engine('sqlite:///sqladb')
connection = engine.connect()
metadata = db.MetaData()
Adult = db.Table('Adult', metadata, autoload=True, autoload_with=engine)

print(Adult.columns.keys())
query = db.select([Adult])

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()


['index', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', '>50K, <=50K']


In [139]:
"""2) Updating"""

df1=pd.DataFrame(ResultSet)
df1.columns = (ResultSet[0].keys())
df1.head(2)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K"
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,51,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [7]:
import pandas as pd

query = db.update(Adult).values(age = 52)
query = query.where(Adult.columns.fnlwgt == 83311)
results = connection.execute(query)
df2 =pd.DataFrame(ResultSet)
df2.columns = (ResultSet[0].keys())
df2.head(2)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K"
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,52,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [11]:
"""3) Delete """

query1 = db.delete(Adult)
query1 = query1.where(Adult.columns.fnlwgt == 77516 )
results = connection.execute(query)
results = connection.execute(db.select([Adult])).fetchall()
df3=pd.DataFrame(results)
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,52,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [16]:
"""4) Filter"""

results_1 = connection.execute(db.select([Adult]).where(Adult.columns.sex == ' Female')).fetchall()
df4=pd.DataFrame(results_1)
df4.columns = (results_1[0].keys())
df4[df4['sex'] == ' Female'].count()

index             10771
age               10771
workclass         10771
fnlwgt            10771
education         10771
education-num     10771
marital-status    10771
occupation        10771
relationship      10771
race              10771
sex               10771
capital-gain      10771
capital-loss      10771
hours-per-week    10771
native-country    10771
>50K, <=50K       10771
dtype: int64

In [23]:
"""5) Function"""

result_func = connection.execute(db.select([db.func.sum(Adult.columns.age)])).fetchall()
df5=pd.DataFrame(result_func)

df5

,0
0,1256218
